In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import dask.dataframe as dd
import warnings
from numba import jit, njit,prange
import psutil

In [2]:
# 경고 무시 설정
warnings.simplefilter(action='ignore', category=FutureWarning)

# 데이터 타입 명시
dtypes = {
    'open': 'float32',
    'high': 'float32',
    'low': 'float32',
    'close': 'float32',
    'volume': 'float32',
    'RSI_10': 'float32',
    'RSI_20': 'float32',
    'RSI_30': 'float32',
    'RSI_40': 'float32',
    'RSI_50': 'float32',
    'RSI_60': 'float32',
    'MACD_Line': 'float32',
    'MACD_Signal': 'float32',
    'MACD_Histogram': 'float32',
    'CCI_5': 'float32',
    'CCI_10': 'float32',
    'CCI_20': 'float32',
    'MFI_10': 'float32',
    'MFI_20': 'float32',
    'MFI_30': 'float32',
    'MFI_40': 'float32',
    'MFI_50': 'float32',
    'MFI_60': 'float32',
    'Volume_Ratio_10': 'float32',
    'Volume_Ratio_20': 'float32',
    'Volume_Ratio_30': 'float32',
    'Volume_Ratio_40': 'float32',
    'Volume_Ratio_50': 'float32',
    'Volume_Ratio_60': 'float32',
    'TR': 'float32',
    'ATR': 'float32',
    'Basic Upper Band': 'float32',
    'Basic Lower Band': 'float32',
    'Final Upper Band': 'float32',
    'Final Lower Band': 'float32',
    'SuperTrend': 'float32',
    'VWAP': 'float32',
    'INDICATOR': 'float32',
    'high_ind': 'float32',
    'low_ind': 'float32',
    'diff': 'float32'
}

# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL60_Indicator.csv'
#file_path = '/content/SOL60_Indicator.csv'

# Dask를 사용하여 CSV 파일 읽기
dask_df = dd.read_csv(file_path, dtype=dtypes, parse_dates=['open_time'])

# Dask 데이터프레임을 pandas 데이터프레임으로 변환
data = dask_df.compute()

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,open_time,open,high,low,close,volume,RSI_10,RSI_20,RSI_30,RSI_40,...,Basic Upper Band,Basic Lower Band,Final Upper Band,Final Lower Band,SuperTrend,VWAP,INDICATOR,high_ind,low_ind,diff
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.339996,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.445267,NaN,NaN,NaN,NaN
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.860001,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,1.446002,NaN,NaN,NaN,NaN
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.129883,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,1.449040,NaN,NaN,NaN,NaN
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.760010,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,1.450617,NaN,NaN,NaN,NaN
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.909973,100.0,100.0,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,1.451432,NaN,NaN,NaN,NaN


In [3]:
columns_list = data.columns.tolist()
columns_list

['open_time',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'RSI_10',
 'RSI_20',
 'RSI_30',
 'RSI_40',
 'RSI_50',
 'RSI_60',
 'MACD_Line',
 'MACD_Signal',
 'MACD_Histogram',
 'CCI_5',
 'CCI_10',
 'CCI_20',
 'MFI_10',
 'MFI_20',
 'MFI_30',
 'MFI_40',
 'MFI_50',
 'MFI_60',
 'Volume_Ratio_10',
 'Volume_Ratio_20',
 'Volume_Ratio_30',
 'Volume_Ratio_40',
 'Volume_Ratio_50',
 'Volume_Ratio_60',
 'TR',
 'ATR',
 'Basic Upper Band',
 'Basic Lower Band',
 'Final Upper Band',
 'Final Lower Band',
 'SuperTrend',
 'VWAP',
 'INDICATOR',
 'high_ind',
 'low_ind',
 'diff']

In [4]:
len(data)

1856536

In [ ]:
@njit
def fast_percentage_change(start, end):
    return ((end - start) / start) * 100

@njit(parallel=True)
def process_data(data, num_features):
    total_length = len(data)
    results = np.empty((total_length, 9 + 3 * num_features), dtype=np.float64)

    for i in prange(total_length):
        start_time = data[i, 0]
        end_time = start_time + 3600  # 60분(1시간) 동안의 초

        subset_indices = np.where((data[:, 0] >= start_time) & (data[:, 0] <= end_time))[0]
        if len(subset_indices) == 0:
            results[i, :] = np.nan
            continue

        subset = data[subset_indices]
        start_close = data[i, 4]

        highest_idx = np.argmax(subset[:, 2])
        lowest_idx = np.argmin(subset[:, 3])

        highest_row = subset[highest_idx]
        lowest_row = subset[lowest_idx]

        highest_change = fast_percentage_change(start_close, highest_row[2])
        lowest_change = fast_percentage_change(start_close, lowest_row[3])

        results[i, 0] = start_time
        results[i, 1] = end_time
        results[i, 2] = data[i, 5]
        results[i, 3] = highest_row[0]
        results[i, 4] = highest_change
        results[i, 5] = highest_row[5]
        results[i, 6] = lowest_row[0]
        results[i, 7] = lowest_change
        results[i, 8] = lowest_row[5]

        start_indicators = data[i, 6:]
        highest_indicators = highest_row[6:]
        lowest_indicators = lowest_row[6:]

        results[i, 9:9 + num_features] = start_indicators  # 시작 지표
        results[i, 9 + num_features:9 + 2 * num_features] = highest_indicators  # 최고 상승 지표
        results[i, 9 + 2 * num_features:] = lowest_indicators  # 최대 하락 지표

    return results

def memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    return mem_info.rss / 1024 ** 2  # MB 단위로 변환

# 데이터 준비
data['open_time'] = pd.to_datetime(data['open_time']).astype(np.int64) // 10**9
data_array = data.to_numpy()
total_length = len(data_array)

# 메모리 사용량 확인
initial_memory_usage = memory_usage()
print(f'Initial memory usage: {initial_memory_usage:.2f} MB')

# 데이터 처리
num_features = data_array.shape[1] - 6

# tqdm을 사용하여 진행 상황 표시
results = np.empty((total_length, 9 + 3 * num_features), dtype=np.float64)

# 데이터 처리 및 진행 상황 표시
for i in tqdm(range(total_length), desc="Processing", ncols=100):
    start_time = data_array[i, 0]
    end_time = start_time + 3600  # 60분(1시간) 동안의 초

    subset_mask = (data_array[:, 0] >= start_time) & (data_array[:, 0] <= end_time)
    subset_indices = np.where(subset_mask)[0]

    if len(subset_indices) == 0:
        results[i, :] = np.nan
        continue

    subset = data_array[subset_indices]
    start_close = data_array[i, 4]

    highest_idx = np.argmax(subset[:, 2])
    lowest_idx = np.argmin(subset[:, 3])

    highest_row = subset[highest_idx]
    lowest_row = subset[lowest_idx]

    highest_change = fast_percentage_change(start_close, highest_row[2])
    lowest_change = fast_percentage_change(start_close, lowest_row[3])

    results[i, 0] = start_time
    results[i, 1] = end_time
    results[i, 2] = data_array[i, 5]
    results[i, 3] = highest_row[0]
    results[i, 4] = highest_change
    results[i, 5] = highest_row[5]
    results[i, 6] = lowest_row[0]
    results[i, 7] = lowest_change
    results[i, 8] = lowest_row[5]

    start_indicators = data_array[i, 6:]
    highest_indicators = highest_row[6:]
    lowest_indicators = lowest_row[6:]

    results[i, 9:9 + num_features] = start_indicators  # 시작 지표
    results[i, 9 + num_features:9 + 2 * num_features] = highest_indicators  # 최고 상승 지표
    results[i, 9 + 2 * num_features:] = lowest_indicators  # 최대 하락 지표

# 결과를 DataFrame으로 변환
start_indicator_columns = [f'start_indicator_{i}' for i in range(num_features)]
highest_indicator_columns = [f'highest_indicator_{i}' for i in range(num_features)]
lowest_indicator_columns = [f'lowest_indicator_{i}' for i in range(num_features)]

column_names = [
    'start_time', 'end_time', 'volume_at_start',
    'highest_time', 'highest_change', 'highest_volume',
    'lowest_time', 'lowest_change', 'lowest_volume'
] + start_indicator_columns + highest_indicator_columns + lowest_indicator_columns

result_df = pd.DataFrame(results, columns=column_names).drop_duplicates().reset_index(drop=True)

# Unix 타임스탬프 열을 datetime으로 다시 변환
for col in ['start_time', 'end_time', 'highest_time', 'lowest_time']:
    result_df[col] = pd.to_datetime(result_df[col], unit='s')

# 메모리 사용량 확인
final_memory_usage = memory_usage()
print(f'Final memory usage: {final_memory_usage:.2f} MB')

Initial memory usage: 1281.91 MB


Processing:   8%|███▏                                   | 150084/1856536 [07:57<1:20:03, 355.23it/s]

In [ ]:
#1시간씩 간격을 나누어서 종가 기준으로 최대 상승 및 하락하는 %를 구하는

@njit
def fast_percentage_change(start, end):
    return ((end - start) / start) * 100

@njit(parallel=True)
def process_data(data, num_features):
    total_length = len(data)
    results = np.empty((total_length, 9 + 3 * num_features), dtype=np.float64)

    for i in prange(total_length):
        start_time = data[i, 0]
        end_time = start_time + 3600  # 60분(1시간) 동안의 초

        subset_mask = (data[:, 0] >= start_time) & (data[:, 0] <= end_time)
        subset = data[subset_mask]

        if len(subset) == 0:
            results[i, :] = np.nan
            continue

        start_close = data[i, 4]

        highest_idx = np.argmax(subset[:, 2])
        lowest_idx = np.argmin(subset[:, 3])

        highest_row = subset[highest_idx]
        lowest_row = subset[lowest_idx]

        highest_change = fast_percentage_change(start_close, highest_row[2])
        lowest_change = fast_percentage_change(start_close, lowest_row[3])

        results[i, 0] = start_time
        results[i, 1] = end_time
        results[i, 2] = data[i, 5]
        results[i, 3] = highest_row[0]
        results[i, 4] = highest_change
        results[i, 5] = highest_row[5]
        results[i, 6] = lowest_row[0]
        results[i, 7] = lowest_change
        results[i, 8] = lowest_row[5]

        start_indicators = data[i, 6:]
        highest_indicators = highest_row[6:]
        lowest_indicators = lowest_row[6:]

        results[i, 9:9 + num_features] = start_indicators  # 시작 지표
        results[i, 9 + num_features:9 + 2 * num_features] = highest_indicators  # 최고 상승 지표
        results[i, 9 + 2 * num_features:] = lowest_indicators  # 최대 하락 지표

    return results

def memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    return mem_info.rss / 1024 ** 2  # MB 단위로 변환

# 데이터 준비
data['open_time'] = pd.to_datetime(data['open_time']).astype(np.int64) // 10**9
data_array = data.to_numpy()

# 메모리 사용량 확인
initial_memory_usage = memory_usage()
print(f'Initial memory usage: {initial_memory_usage:.2f} MB')

# 데이터 처리
num_features = data_array.shape[1] - 6

# tqdm을 사용하여 진행 상황 표시
results = np.empty((len(data_array), 9 + 3 * num_features), dtype=np.float64)

# 데이터 처리 및 진행 상황 표시
for i in tqdm(prange(len(data_array)), desc="Processing", ncols=100):
    start_time = data_array[i, 0]
    end_time = start_time + 3600  # 60분(1시간) 동안의 초

    subset_mask = (data_array[:, 0] >= start_time) & (data_array[:, 0] <= end_time)
    subset_indices = np.where(subset_mask)[0]

    if len(subset_indices) == 0:
        results[i, :] = np.nan
        continue

    subset = data_array[subset_indices]
    start_close = data_array[i, 4]

    highest_idx = np.argmax(subset[:, 2])
    lowest_idx = np.argmin(subset[:, 3])

    highest_row = subset[highest_idx]
    lowest_row = subset[lowest_idx]

    highest_change = fast_percentage_change(start_close, highest_row[2])
    lowest_change = fast_percentage_change(start_close, lowest_row[3])

    results[i, 0] = start_time
    results[i, 1] = end_time
    results[i, 2] = data_array[i, 5]
    results[i, 3] = highest_row[0]
    results[i, 4] = highest_change
    results[i, 5] = highest_row[5]
    results[i, 6] = lowest_row[0]
    results[i, 7] = lowest_change
    results[i, 8] = lowest_row[5]

    start_indicators = data_array[i, 6:]
    highest_indicators = highest_row[6:]
    lowest_indicators = lowest_row[6:]

    results[i, 9:9 + num_features] = start_indicators  # 시작 지표
    results[i, 9 + num_features:9 + 2 * num_features] = highest_indicators  # 최고 상승 지표
    results[i, 9 + 2 * num_features:] = lowest_indicators  # 최대 하락 지표

# 결과를 DataFrame으로 변환
start_indicator_columns = [f'start_indicator_{i}' for i in range(num_features)]
highest_indicator_columns = [f'highest_indicator_{i}' for i in range(num_features)]
lowest_indicator_columns = [f'lowest_indicator_{i}' for i in range(num_features)]

column_names = [
    'start_time', 'end_time', 'volume_at_start',
    'highest_time', 'highest_change', 'highest_volume',
    'lowest_time', 'lowest_change', 'lowest_volume'
] + start_indicator_columns + highest_indicator_columns + lowest_indicator_columns

result_df = pd.DataFrame(results, columns=column_names).drop_duplicates().reset_index(drop=True)

# Unix 타임스탬프 열을 datetime으로 다시 변환
for col in ['start_time', 'end_time', 'highest_time', 'lowest_time']:
    result_df[col] = pd.to_datetime(result_df[col], unit='s')

# 메모리 사용량 확인
final_memory_usage = memory_usage()
print(f'Final memory usage: {final_memory_usage:.2f} MB')

Initial memory usage: 1279.45 MB


Processing:   1%|▍                                       | 19119/1856536 [01:15<1:53:26, 269.94it/s]

In [ ]:
result_df.head()

In [ ]:
len(result_df)

In [ ]:
# 저장할 경로 설정
save_path1 = '/content/drive/MyDrive/Data/SOL60_Indicator_rise_fall_percentages.csv'

# CSV 파일로 저장
result_df.to_csv(save_path1, index=False)

In [ ]:
file_path2 = '/content/sample_data/SOL60_Indicator_rise_fall_percentages.csv'
data2 = pd.read_csv(file_path2)

In [ ]:
import ast

In [ ]:
# start_time을 시-분 형식으로 변환
data2['time'] = pd.to_datetime(data2['start_time']).dt.strftime('%H:%M')

# start_indicators를 데이터프레임으로 변환
start_indicators_cols = ['RSI_10', 'RSI_20', 'RSI_60', 'MACD_Line', 'Signal_Line', 'MACD_Histogram', 'CCI_5',
                         'CCI_10', 'CCI_20', 'RSI_Stoch_K_10', 'RSI_Stoch_D_10', 'RSI_Stoch_K_20', 'RSI_Stoch_D_20',
                         'RSI_Stoch_K_60', 'RSI_Stoch_D_60', 'MFI_10', 'MFI_20', 'MFI_60', 'Volume_Ratio_10',
                         'Volume_Ratio_20', 'Volume_Ratio_60', 'BB_Mean_10', 'BB_Upper_10', 'BB_Lower_10', 'BB_Mean_20',
                         'BB_Upper_20', 'BB_Lower_20', 'BB_Mean_60', 'BB_Upper_60', 'BB_Lower_60', 'Tenkan_Sen',
                         'Kijun_Sen', 'Senkou_Span_A', 'Senkou_Span_B', 'Chikou_Span', 'Envelope_MA_10',
                         'Envelope_Upper_10', 'Envelope_Lower_10', 'Envelope_MA_20', 'Envelope_Upper_20',
                         'Envelope_Lower_20', 'Envelope_MA_60', 'Envelope_Upper_60', 'Envelope_Lower_60', 'INDICATOR',
                         'high_ind', 'low_ind', 'diff']

# start_indicators를 분리하여 열로 변환
def parse_indicators(indicator_str):
    try:
        return list(map(float, indicator_str.strip('[]').split(', ')))
    except (ValueError, SyntaxError):
        return [None] * len(start_indicators_cols)

start_indicators = data2['start_indicators'].apply(parse_indicators)
start_indicators_df = pd.DataFrame(start_indicators.tolist(), columns=start_indicators_cols)

# data2에 start_indicators 열 추가
data2 = pd.concat([data2, start_indicators_df], axis=1)

# highest_change가 1.2 이상인 데이터만 필터링
filtered_data2 = data2[data2['highest_change'] >= 1.2]

# 그룹화 및 통계 계산 (highest_change가 1.2 이상인 데이터에 대해)
grouped_filtered = filtered_data2.groupby('time').agg({
    'highest_change': ['max', 'mean'],
    'lowest_change': ['min', 'mean'],
    **{col: 'mean' for col in start_indicators_cols}
})

# 전체 그룹에 대한 1.2% 이상 상승할 확률 계산
def calc_probability(group):
    return (group['highest_change'] >= 1.2).mean() * 100

probabilities = data2.groupby('time').apply(calc_probability).reset_index()
probabilities.columns = ['time', 'probability_above_1.2']

# 결과 병합
grouped_filtered.columns = ['_'.join(col).strip() for col in grouped_filtered.columns.values]
results_filtered = grouped_filtered.reset_index()
results_filtered = results_filtered.merge(probabilities, on='time', how='left')

# 결과 출력
results.head()

,time,max_highest_change,mean_highest_change,max_lowest_change,mean_lowest_change,probability_above_1.2,mean_RSI_10,mean_RSI_20,mean_RSI_60,mean_MACD_Line,...,mean_Envelope_MA_20,mean_Envelope_Upper_20,mean_Envelope_Lower_20,mean_Envelope_MA_60,mean_Envelope_Upper_60,mean_Envelope_Lower_60,mean_INDICATOR,mean_high_ind,mean_low_ind,mean_diff
1423,23:43,5.905056,2.348141,-1.196449,-0.542188,85.714286,56.265832,46.805774,50.885138,-0.003573,...,24.628429,25.244139,24.012718,24.636881,25.252803,24.020959,56.265832,85.345399,11.666541,73.678858
1424,23:44,5.741811,2.271935,-1.348748,-0.614915,85.714286,63.022406,48.730399,51.331594,0.000528,...,24.629143,25.244871,24.013414,24.637643,25.253584,24.021702,63.022406,86.474601,12.359182,74.115419
1425,23:45,5.905056,2.355389,-1.196449,-0.522173,85.714286,62.626041,49.405706,51.275089,0.002026,...,24.629571,25.245311,24.013832,24.638286,25.254243,24.022329,62.626041,87.278172,12.618922,74.659250
1439,23:59,23.529412,4.913316,-1.576317,-0.771817,71.428571,54.322755,55.535669,52.316853,0.016706,...,24.668286,25.284993,24.051579,24.649690,25.265933,24.033448,54.322755,89.040695,15.237970,73.802725
1426,23:46,10.810811,3.066347,-1.158301,-0.507067,71.428571,61.423160,47.460200,51.180686,0.002834,...,24.629143,25.244871,24.013414,24.638833,25.254804,24.022862,61.423160,87.278172,13.008533,74.269639


In [ ]:
results = results.sort_values(by='probability_above_1.2', ascending=False)
results.head()

,time,max_highest_change,mean_highest_change,max_lowest_change,mean_lowest_change,probability_above_1.2,mean_RSI_10,mean_RSI_20,mean_RSI_60,mean_MACD_Line,...,mean_Envelope_MA_20,mean_Envelope_Upper_20,mean_Envelope_Lower_20,mean_Envelope_MA_60,mean_Envelope_Upper_60,mean_Envelope_Lower_60,mean_INDICATOR,mean_high_ind,mean_low_ind,mean_diff
1423,23:43,5.905056,2.348141,-1.196449,-0.542188,85.714286,56.265832,46.805774,50.885138,-0.003573,...,24.628429,25.244139,24.012718,24.636881,25.252803,24.020959,56.265832,85.345399,11.666541,73.678858
1425,23:45,5.905056,2.355389,-1.196449,-0.522173,85.714286,62.626041,49.405706,51.275089,0.002026,...,24.629571,25.245311,24.013832,24.638286,25.254243,24.022329,62.626041,87.278172,12.618922,74.659250
1424,23:44,5.741811,2.271935,-1.348748,-0.614915,85.714286,63.022406,48.730399,51.331594,0.000528,...,24.629143,25.244871,24.013414,24.637643,25.253584,24.021702,63.022406,86.474601,12.359182,74.115419
1427,23:47,13.418407,3.427324,-1.005414,-0.496798,71.428571,61.710457,48.151675,50.062148,0.003094,...,24.628500,25.244212,24.012788,24.638786,25.254755,24.022816,61.710457,87.278172,13.008533,74.269639
1438,23:58,23.339731,4.869717,-1.727447,-0.809555,71.428571,59.867347,58.406131,52.541462,0.016817,...,24.664429,25.281039,24.047818,24.648357,25.264566,24.032148,59.867347,89.040695,12.618922,76.421773


In [ ]:
# 저장할 경로 설정
save_path2 = '/content/sample_data/SOL60_1.2_probability.csv'

# CSV 파일로 저장
results.to_csv(save_path2, index=False)

In [ ]:
# probability_above_1.2가 70 이상인 항목 필터링
filtered_results = results[results['probability_above_1.2'] >= 70]
filtered_results = filtered_results.sort_values(by='time', ascending=False)
filtered_results

,time,max_highest_change,mean_highest_change,max_lowest_change,mean_lowest_change,probability_above_1.2,mean_RSI_10,mean_RSI_20,mean_RSI_60,mean_MACD_Line,...,mean_Envelope_MA_20,mean_Envelope_Upper_20,mean_Envelope_Lower_20,mean_Envelope_MA_60,mean_Envelope_Upper_60,mean_Envelope_Lower_60,mean_INDICATOR,mean_high_ind,mean_low_ind,mean_diff
1439,23:59,23.529412,4.913316,-1.576317,-0.771817,71.428571,54.322755,55.535669,52.316853,0.016706,...,24.668286,25.284993,24.051579,24.649690,25.265933,24.033448,54.322755,89.040695,15.237970,73.802725
1438,23:58,23.339731,4.869717,-1.727447,-0.809555,71.428571,59.867347,58.406131,52.541462,0.016817,...,24.664429,25.281039,24.047818,24.648357,25.264566,24.032148,59.867347,89.040695,12.618922,76.421773
1437,23:57,23.056300,4.803835,-1.953275,-0.866569,71.428571,64.749757,61.716160,52.697523,0.015841,...,24.659643,25.276134,24.043152,24.646929,25.263102,24.030755,64.749757,89.040695,12.618922,76.421773
1436,23:56,23.624471,4.779179,-1.500577,-0.769400,71.428571,62.643443,60.185818,52.309350,0.012911,...,24.653643,25.269984,24.037302,24.645452,25.261589,24.029316,62.643443,89.040695,12.618922,76.421773
1435,23:55,23.576923,4.817196,-1.538462,-0.730941,71.428571,59.811534,60.044218,51.676358,0.011601,...,24.648786,25.265005,24.032566,24.644286,25.260393,24.028179,59.811534,89.040695,12.618922,76.421773
1434,23:54,23.672055,4.837445,-1.462664,-0.729164,71.428571,55.571564,58.575382,51.739898,0.010778,...,24.643643,25.259734,24.027552,24.643310,25.259392,24.027227,55.571564,89.040695,12.618922,76.421773
1433,23:53,23.576923,4.848131,-1.538462,-0.716491,71.428571,56.492186,55.976667,51.400862,0.010008,...,24.639357,25.255341,24.023373,24.642286,25.258343,24.026229,56.492186,89.040695,12.618922,76.421773
1432,23:52,23.672055,4.847511,-1.462664,-0.719551,71.428571,56.134592,55.219329,51.383117,0.009316,...,24.636143,25.252046,24.020239,24.641476,25.257513,24.025439,56.134592,87.482254,12.618922,74.863332
1431,23:51,23.576923,4.865723,-1.538462,-0.673902,71.428571,55.517012,54.484230,51.068481,0.008152,...,24.633143,25.248971,24.017314,24.640786,25.256805,24.024766,55.517012,87.482254,12.618922,74.863332
1430,23:50,23.815029,4.922474,-1.348748,-0.624784,71.428571,54.791558,51.533538,51.135835,0.007128,...,24.630357,25.246116,24.014598,24.640262,25.256268,24.024255,54.791558,87.482254,12.618922,74.863332


In [ ]:
len(filtered_results)

26